In [ ]:
""" 
1- Preciso excluir as linhas de 'nível' 'SECUNDARIA'. Acabei mandando a base toda pra rodar. 
"""

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

df_main = pd.read_csv('./csv/ESTANDAR_BASICA_I2324.csv')
df_scraping = pd.read_csv('./csv/df_scraping.csv')
extraction = pd.read_csv('./csv/extraction.csv')

In [ ]:
# Verificar quantos números foram adicionados através da contagem de ocorrências válidas para a coluna 'telefone_scraping'.

tel_adicionados = df_scraping[df_scraping['telefone_scraping'].notna()]   
porcentagem = (len(tel_adicionados) / len(df_scraping)) * 100 
print(f'O dataframe tem {len(df_scraping)} linhas.')                    
print(f'Foram adicionados {len(tel_adicionados)} números de telefone novos ({porcentagem:.1f}%).')

O dataframe tem 231534 linhas.
Foram adicionados 25517 números de telefone novos (11.0%).


In [13]:
import phonenumbers

# Cria uma cópia para evitar warning
tel_validos = df_scraping[df_scraping['telefone_scraping'].notna()].copy()

def validar_telefone(numero, codigo_pais='MX'):
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        return phonenumbers.is_valid_number(parsed)
    except phonenumbers.NumberParseException:
        return False

# Aplica a validação de forma segura
tel_validos['telefone_valido'] = tel_validos['telefone_scraping'].apply(
    lambda x: validar_telefone(x, 'MX')
)

# Exibe os resultados
print(f'Total de telefones adicionados: {len(tel_validos)}')
print("Total de telefones válidos:", tel_validos['telefone_valido'].sum())

proporcao = tel_validos['telefone_valido'].mean() * 100
print(f"Proporção de válidos em relação a inválidos: {proporcao:.1f}%")

telefones_invalidos = tel_validos[~tel_validos['telefone_valido']]
print(f'Quantidade de telefones inválidos: {len(telefones_invalidos)}')

Total de telefones adicionados: 25517
Total de telefones válidos: 15084
Proporção de válidos em relação a inválidos: 59.1%
Quantidade de telefones inválidos: 10433


In [15]:
print('Telefones inválidos abaixo:')
display(telefones_invalidos['telefone_scraping'])

Telefones inválidos abaixo:


60                 9132982
66                 9967317
88                 9725177
124                9939640
151           616 111 81 4
168                1407431
333                9145044
358           016642044183
364                1940450
405              646211593
472                9762385
493                5923294
499           01658 517323
514                9564944
532              686135169
540                9637400
573                9639494
664                9783708
736                9949153
929              449722310
930                9148778
1000               5486533
1036               9122273
1075             777721375
1116               1031636
1140               9123262
1231                   646
1274               5582480
1366             686520446
1395               3778021
1439               7625995
1529             664851752
1559               7620163
1648             686568129
1865               7720388
2004               9739613
2192               7720891
2

In [16]:
# Filtra os inválidos
telefones_invalidos = tel_validos[~tel_validos['telefone_valido']].copy()

# Cria coluna com o comprimento dos números
telefones_invalidos['qtde_caracteres'] = telefones_invalidos['telefone_scraping'].str.len()

# Agrupa e conta
resumo = telefones_invalidos.groupby('qtde_caracteres').size().reset_index(name='quantidade')

display(resumo)


,qtde_caracteres,quantidade
0,1,2
1,3,2
2,4,184
3,5,5
4,6,1
5,7,1782
6,8,6434
7,9,46
8,10,18
9,11,13
